# Análisis Predictivo: Fuga de Clientes en Telecomunicaciones

Este cuaderno tiene como objetivo explorar un caso real de predicción de fuga de clientes usando un árbol de decisión.

## Objetivo
Reflexionar sobre el uso de modelos predictivos y prescriptivos para tomar decisiones empresariales.

📌 **Nota:** No es necesario programar. Simplemente ejecuta las celdas y reflexiona sobre las preguntas propuestas.

In [ ]:
# Cargar librerías
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Cargar datos desde GitHub
url = "https://raw.githubusercontent.com/taruntiwarihp/dataSets/master/telecom_churn.csv"
df = pd.read_csv(url)
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## ¿Qué representan estos datos?
- Cada fila representa un cliente.
- Las columnas muestran información sobre su uso de servicios, gastos y si se dio de baja (`Churn`).

### 💬 Reflexión:
¿Qué variables crees que podrían influir más en que un cliente se dé de baja?

####  ¿Qué variables crees que podrían influir más en que un cliente se dé de baja?

Una de las variables más importantes es el número de **llamadas al servicio de atención al cliente**, ya que si una persona llama muchas veces puede significa que está teniendo problemas que no se han resuelto. Esto es un buen motivo que puede llevar a que se canse y se dé de baja.

Otra variable que puede influir es si el cliente tiene **plan internacional** o **plan de buzón de voz**. Si no tiene estos servicios, puede ser que la empresa no le esté ofreciendo algo útil o atractivo, y por eso se va.

También influye cuánto **tiempo habla al día** y cuántas **llamadas hace**. Si un cliente usa mucho los servicios y de repente baja su uso, es indicativo de que algo pasa con ese cliente, incluso podría ser una señal de que está pensando en darse de baja.


In [ ]:
# Preprocesamiento mínimo (automático)
df.drop(["State", "Area code", "Phone"], axis=1, inplace=True, errors='ignore')
df["Churn"] = df["Churn"].astype(int)
df = pd.get_dummies(df, drop_first=True)

X = df.drop("Churn", axis=1)
y = df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento y predicción con árbol de decisión
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       566
           1       0.88      0.67      0.76       101

    accuracy                           0.94       667
   macro avg       0.91      0.83      0.86       667
weighted avg       0.93      0.94      0.93       667



### 💬 Reflexión:
- ¿El modelo predice bien los clientes que se dan de baja?
- ¿Qué errores comete? ¿Qué consecuencias tendría para la empresa?
- ¿Preferirías minimizar los falsos positivos o los falsos negativos?

####  ¿El modelo predice bien los clientes que se dan de baja?

El modelo acierta bastante en general, con un **94% de precisión total**, pero cuando miramos más en detalle, vemos que **detecta mejor a los que no se dan de baja** (precisión del 94%, *recall* del 98%) que a los que **sí se dan de baja** (precisión del 88%, pero solo *recall* del 67%).

En resumen, aunque el modelo es bastante bueno, en general **se le escapan bastantes clientes que sí se dan de baja**, y eso puede ser un problema si lo que queremos es evitar que se vayan. Digamos que el modelo sirve como punto de partida, pero se podría mejorar para detectar mejor a los clientes que realmente están en riesgo de irse.

---

####  ¿Qué errores comete? ¿Qué consecuencias tendría para la empresa?

El modelo comete dos tipos de errores:

- **Falsos negativos**: Son clientes que sí se van, pero el modelo dice que no. Este es el error más grave porque la empresa **no haría nada para retenerlos**, y acabaría perdiéndolos sin darse cuenta.

- **Falsos positivos**: Son clientes que no se van, pero el modelo cree que sí. En este caso, la empresa **podría gastar recursos innecesarios** (por ejemplo, haciendo descuentos o llamadas de fidelización a gente que no lo necesitaba).

La consecuencia más importante es que si se pierden muchos clientes sin detectarlos, **baja la rentabilidad de la empresa**. Y si se gastan recursos mal, también **se pierde dinero en estrategias mal enfocadas**.

---

####  ¿Preferirías minimizar los falsos positivos o los falsos negativos?

Yo preferiría **minimizar los falsos negativos**, porque esos son los clientes que realmente se van, pero el modelo no los detecta. Si no los identificamos, **la empresa no hace nada para retenerlos** y se pierden sin avisar.

Aunque los falsos positivos también son un problema (se gasta dinero innecesariamente), pero es **más importante no dejar escapar a los clientes que están en riesgo**, ya que recuperarlos después puede ser mucho más difícil y caro.



## De la predicción a la acción: Análisis prescriptivo

Supón que detectas que clientes con muchas llamadas al servicio de atención tienden a darse de baja.

### 💬 Reflexión:
- ¿Qué acción podrías recomendar a la empresa?
- ¿Cómo podrías personalizar la oferta o el servicio?

Este tipo de análisis ayuda a mejorar la toma de decisiones estratégicas con base en los datos.

#### ¿Qué acción podrías recomendar a la empresa? ¿Cómo podrías personalizar la oferta o el servicio?

Una propuesta de accion sería **contactar con los clientes que han hecho muchas llamadas al servicio de atención al cliente**, con el fin de ofrecerles una ayuda especial o bien un trato preferente. Esto les demostraría que la empresa se preocupa por ellos y así evitar que se vayan.

Sería conveniente realizar **ofertas personalizadas** según el uso que haga cada cliente. Por ejemplo, si un cliente habla mucho por las noches, se le puede ofrecer un **plan nocturno más barato**. O si tiene muchos minutos internacionales, se le podría dar un **descuento en llamadas internacionales**.

La idea es usar los datos que tenemos de los clientes, para **adaptar las ofertas a las necesidades del cliente**, personalizando las ofertas, en lugar de ofrecer lo mismo a todos.